In [ ]:
!pip install transformers sentence-transformers langchain faiss-cpu pypdf langchain-community

In [ ]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

In [ ]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

def create_vector_database(text, model_name="sentence-transformers/all-MiniLM-L6-v2"):
    sentences = text.split(". ")
    model = SentenceTransformer(model_name)

    embeddings = model.encode(sentences, show_progress_bar=True)

    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(np.array(embeddings))
    return index, sentences

In [ ]:
def retrieve_answer(query, index, sentences, model_name="sentence-transformers/all-MiniLM-L6-v2"):
    model = SentenceTransformer(model_name)
    query_embedding = model.encode([query])

    D, I = index.search(query_embedding, k=5)
    results = [sentences[i] for i in I[0]]
    return results

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def generate_answer(context, query, model_name="t5-small"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    input_text = f"question: {query} context: {context}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs["input_ids"], max_length=100, num_beams=4, early_stopping=True)

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [ ]:
def chat_with_pdf(pdf_path, query):
    text = extract_text_from_pdf(pdf_path)

    index, sentences = create_vector_database(text)

    retrieved_context = " ".join(retrieve_answer(query, index, sentences))

    answer = generate_answer(retrieved_context, query)
    return answer

In [ ]:
pdf_path = "file path"
query = "What is the topic of this document?"

answer = chat_with_pdf(pdf_path, query)
print("Answer:", answer)